<a href="https://colab.research.google.com/github/martinovzky/Pytorch-Classifier/blob/main/Pytorch_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries, Hyperparameters

In [5]:
#libraries

import torch as torch
import torch.nn as nn         # classes and functions for NN building
import torch.optim as optim   # optimization algorithms for training
from torch.utils.data import Dataset, DataLoader
import torchvision            # pre-trained models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder # for batching and loading data
import timm                                  # contains the models

import matplotlib.pyplot as plt              # for data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [14]:
#hyperparameters

batch = 32               # of images / training batch
epochs = 10              # tot number of training epochs (full passes over the dataset)
LearningRate = 1e-3      # learning rate for the optimizer (how much should model weights be updated after each batch )
NumClasses = 67          # MIT‑67 dataset has 67 scene categories
device = "cuda" if torch.cuda.is_available() else "cpu"


# Data Preparation


In [23]:
import zipfile
import os
from google.colab import drive

drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/Colab_Notebooks/train'
test_dir = '/content/drive/MyDrive/Colab_Notebooks/test'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# transformations for training and validation

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),             # resizes images to 224x224 pixels
    transforms.RandomHorizontalFlip(),         # randomly flips images horizontally for data augmentation
    transforms.ToTensor(),                     # converts PIL Image to a PyTorch tensor (scales pixels to [0,1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # normalizes using ImageNet statistics
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),             # resizes images for consistency
    transforms.ToTensor(),                     # converts images to tensor, pixel values now between [0,1].
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # uses same normalization as training
                         std=[0.229, 0.224, 0.225])
])


# assigns labels to images based on subfolder names (= class names)
train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = ImageFolder(root=test_dir, transform=val_transforms)

# data loaders to handle batching and shuffling
train_loader = DataLoader(train_dataset, batch_size= batch, shuffle=True, num_workers=2) # shuffle in order not to get overfitting
val_loader   = DataLoader(val_dataset, batch_size= batch, shuffle=False, num_workers=2) #num_workers = 2: 2 subprocesses to speed things up


# Model Setup


In [26]:
model = timm.create_model("efficientnet_b0", pretrained=True)

#adapts the model to classify images into NumClasses instead of the default 1000 ImageNet categories

in_features = model.classifier.in_features             # number of input features for the classifier layer
model.classifier = nn.Linear(in_features, NumClasses)  # replaces with a new linear layer

model = model.to(device)